# file_system

In [ ]:
open rust_operators
open sm'_operators

()



In [ ]:
// // test

open testing

()



## types

In [ ]:
inl types () =
    ()

()



## (</>)

In [ ]:
inl (</>) (a : string) (b : string) : string =
    $'System.IO.Path.Combine (!a, !b)'

()



## create_temp_directory_name

In [ ]:
inl create_temp_directory_name () =
    run_target function
        | Fsharp (Native) => fun () =>
            inl root = $'System.Reflection.Assembly.GetEntryAssembly().GetName().Name' : string

            ($'System.IO.Path.GetTempPath' () : string)
            </> ($'$"{!root}"' : string)
            </> (date_time.new_guid_from_date_time $'System.DateTime.Now' |> sm'.obj_to_string)
        | x => fun () => failwith $'$"create_temp_directory_name target: {!x}"'

()



In [ ]:
// // test

create_temp_directory_name ()
|> _contains ($'System.IO.Path.DirectorySeparatorChar' : char)

type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1 of f1_0 : US0
and [<Struct>] US2 =
    | US2_0
    | US2_1 of f1_0 : char
let rec closure0 (v0 : char) (v1 : char) : bool =
    let v2 : bool = v0 = v1
    v2
and closure1 () (v0 : char) : US2 =
    US2_1(v0)
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let mutable result = None
    #if FABLE_COMPILER_RUST && !WASM
    let v0 : US0 = US0_1
    let v1 : US1 = US1_1(v0)
    let v2 : string = $"create_temp_directory_name target: {v1}"
    let v3 : string = failwith<string> v2
    v3
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v4 : US0 = US0_2
    let v5 : US1 = US1_1(v4)
    let v6 : string = $"create_temp_directory_name target: {v5}"
    let v7 : string = failwith<string> v6
    v7
    #endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !WASM
    let v8 : string = System.Reflection.Assembly.GetEntryAssembly().GetName().Name
    let v

## main

In [ ]:
inl main () =
    $"let create_temp_directory_name () = !create_temp_directory_name ()" : ()

type [<Struct>] US0 =
    | US0_0
    | US0_1
    | US0_2
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1 of f1_0 : US0
let rec closure0 () () : string =
    let mutable result = None
    #if FABLE_COMPILER_RUST && !WASM
    let v0 : US0 = US0_1
    let v1 : US1 = US1_1(v0)
    let v2 : string = $"create_temp_directory_name target: {v1}"
    let v3 : string = failwith<string> v2
    v3
    #endif
    #if FABLE_COMPILER_RUST && WASM
    let v4 : US0 = US0_2
    let v5 : US1 = US1_1(v4)
    let v6 : string = $"create_temp_directory_name target: {v5}"
    let v7 : string = failwith<string> v6
    v7
    #endif
    #if !FABLE_COMPILER && !FABLE_COMPILER_RUST && !WASM
    let v8 : string = System.Reflection.Assembly.GetEntryAssembly().GetName().Name
    let v9 : (unit -> string) = System.IO.Path.GetTempPath
    let v10 : string = v9 ()
    let v11 : string = $"{v8}"
    let v12 : string = System.IO.Path.Combine (v10, v11)
    let v13 : System.DateTime = System.DateTime.Now
    le